<a href="https://colab.research.google.com/github/entanglement-nak/portfolio-nak/blob/main/%E3%83%9D%E3%83%BC%E3%83%88%E3%83%95%E3%82%A9%E3%83%AA%E3%82%AA%E3%80%80lightbgm%E4%BD%BF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# データパス
train_path = r"/content/drive/MyDrive/Wafer/Wafer_TRAIN.xlsx"

test_path = r"/content/drive/MyDrive/Wafer/Wafer_TEST.xlsx"

In [4]:
train_df = pd.read_excel(train_path, header=None)

test_df = pd.read_excel(test_path, header=None)

In [5]:
# トレインデータとテストデータを縦に統合（スタック）
combined_data = pd.concat([train_df, test_df], axis=0)
combined_data.head()

,0,1,2,3,4,5,6,7,8,9,...,143,144,145,146,147,148,149,150,151,152
0,1,-1.602294,-1.670823,-1.693666,-1.699377,-1.699377,-1.703660,-1.703660,-1.703660,-1.703660,...,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432
1,1,1.084591,1.084591,1.084591,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,...,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308
2,1,0.362689,0.362689,0.362689,0.393316,0.362689,0.362689,0.362689,0.362689,0.362689,...,0.393316,0.393316,0.393316,0.362689,0.393316,0.393316,0.393316,0.393316,0.393316,0.393316
3,1,-1.094523,-1.094523,-1.094523,-1.096732,-1.094523,-1.096732,-1.094523,-1.094523,-1.094523,...,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732
4,1,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,...,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761


In [6]:
# データを特徴量(X)とターゲット(y)に分割
X = combined_data.drop(combined_data.columns[0], axis=1)
y = combined_data[combined_data.columns[0]]

In [7]:
X

,1,2,3,4,5,6,7,8,9,10,...,143,144,145,146,147,148,149,150,151,152
0,-1.602294,-1.670823,-1.693666,-1.699377,-1.699377,-1.703660,-1.703660,-1.703660,-1.703660,-1.703660,...,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432,-1.145432
1,1.084591,1.084591,1.084591,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.084591,...,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308,1.065308
2,0.362689,0.362689,0.362689,0.393316,0.362689,0.362689,0.362689,0.362689,0.362689,0.362689,...,0.393316,0.393316,0.393316,0.362689,0.393316,0.393316,0.393316,0.393316,0.393316,0.393316
3,-1.094523,-1.094523,-1.094523,-1.096732,-1.094523,-1.096732,-1.094523,-1.094523,-1.094523,-1.096732,...,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732,-1.096732
4,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,...,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761,-1.103761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6159,1.058364,1.058364,1.077259,1.058364,1.077259,1.077259,1.058364,1.058364,1.077259,1.077259,...,1.058364,1.058364,1.058364,1.058364,1.058364,1.058364,1.058364,1.058364,1.058364,1.058364
6160,-0.883897,-1.037149,-0.979977,-1.012094,-0.992098,-1.004760,-1.000370,-0.996084,-1.011200,-0.981698,...,-0.999874,-1.000244,-0.998784,-1.001801,-0.996731,-1.004760,-0.992098,-1.012094,-0.979977,-1.037149
6161,0.700052,0.874868,0.836440,0.798761,0.786091,0.856247,0.849871,0.772646,0.900171,0.763408,...,0.780763,0.781052,0.779912,0.782268,0.778308,0.784578,0.774691,0.790305,0.765226,0.809869
6162,-1.517047,-1.580021,-1.602920,-1.608645,-1.608645,-1.608645,-1.608645,-1.608645,-1.608645,-1.612939,...,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059,-1.059059


In [8]:
y

0       1
1       1
2       1
3       1
4       1
       ..
6159    1
6160   -1
6161   -1
6162    1
6163    1
Name: 0, Length: 7164, dtype: int64

In [9]:
# データを訓練用とテスト用に7:3の比率で分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# 訓練データ内で正常と異常のクラスを分離し、異常クラスのアップサンプリングを行う
df_train = pd.concat([X_train, y_train], axis=1)
df_normal_train = df_train[df_train[df_train.columns[-1]] == 1]
df_anomaly_train = df_train[df_train[df_train.columns[-1]] == -1]

df_anomaly_upsampled = resample(df_anomaly_train, replace=True, n_samples=len(df_normal_train), random_state=123)
df_upsampled_train = pd.concat([df_normal_train, df_anomaly_upsampled])

In [11]:
# アップサンプリングされたデータを特徴量(X)とターゲット(y)に分割
X_train_upsampled = df_upsampled_train.drop(df_upsampled_train.columns[-1], axis=1)
y_train_upsampled = df_upsampled_train[df_upsampled_train.columns[-1]]

In [12]:
# LightGBM分類器の初期化
lgbm = LGBMClassifier()

# モデルのトレーニング
lgbm.fit(X_train_upsampled, y_train_upsampled)

# テストセットに対する予測
y_pred = lgbm.predict(X_test)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
MCC = matthews_corrcoef(y_test, y_pred)
F1 = f1_score(y_test, y_pred, average='binary')
report = classification_report(y_test, y_pred)

print("\n")
print(f"正解率：{accuracy}")
print(f"MCC：{MCC}")
print(f"F1：{F1}")

[LightGBM] [Info] Number of positive: 4468, number of negative: 4468
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38759
[LightGBM] [Info] Number of data points in the train set: 8936, number of used features: 152
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


正解率：0.9962790697674418
MCC：0.9796322325802044
F1：0.9979296066252588
